<a href="https://colab.research.google.com/github/net39/ML-anomaly-detection/blob/main/P2_05_CNN1_img.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Phase:** CNN1 Image Classification Refinement  
**GPU:** NVIDIA A100

---
**Objective**

This notebook refines the earlier SVM classification results by applying a Convolutional Neural Network (CNN) using the

---
**VGG19 pretrained model**.

The goals are:
- Improve action classification accuracy on test images.  
- Introduce a new 'type' label (normal/abnormal) based on predefined action mappings.  
- Use **ImageNet predictions via VGG19** for cases where the CNN1 classifier is uncertain, identifying possible new classes.
- Add a 'status' column to indicate whether a label is from the existing 15-class set or newly detected.
---

| Path | Description |
|:---------------------------|:--------------------------------|
| `processed/svm_final_img/images/` | Final still images from SVM phase |
| `processed/svm_final_img/svm_final_metadatav2.csv` | Metadata with image labels and sources |
| `metadata/cnn1_final_metadata.csv` | New metadata with refined labels, type, status |
| `models/cnn1_vgg19_model.h5` | Saved CNN1 (VGG19-based) model |

---

**Model Summary**

- **Base Model:** VGG19 (ImageNet weights, no top layers)  
- **Classifier:**  
  - Flatten  
  - Dense (256, ReLU)  
  - Dropout (0.5)  
  - Dense (Softmax, 15 classes)  
- **Loss Function:** Categorical Crossentropy  
- **Optimizer:** Adam  
- **Evaluation:** Prediction confidence threshold (≥ 0.5); otherwise fallback to ImageNet decoding.

---
**Next Step**

The refined labels and metadata will be used in **CNN2 + LSTM video frame classification** for temporal behaviour detection and abnormal activity analysis.


# Install Libraries and Google Access

In [16]:
!nvidia-smi


Sun Jun 22 16:11:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             50W /  400W |    2739MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [31]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [17]:
!pip install --extra-index-url=https://pypi.nvidia.com cuml-cu12 --upgrade
!pip install --quiet google-cloud-storage opencv-python-headless joblib scikit-learn tensorflow

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com


In [1]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/drive')

from google.colab import files
files.upload()

from google.cloud import storage
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/p2-anomaly-c4545180e308.json'

storage_client = storage.Client()
bucket_name = 'p2-anomaly'
bucket = storage_client.bucket(bucket_name)

Mounted at /content/drive


Saving p2-anomaly-c4545180e308.json to p2-anomaly-c4545180e308.json


In [18]:
import cv2
import time
import io
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

from PIL import Image
from io import BytesIO, StringIO
from tqdm import tqdm

# TensorFlow and VGG19 Preprocessing
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout

# Load Metadata

In [19]:
metadata_blob = bucket.blob('processed/svm_final_img/svm_final_metadatav2.csv')
metadata_df = pd.read_csv(StringIO(metadata_blob.download_as_text()))

metadata_df.head()

# Define mapping of action labels to type (normal / abnormal)
label_type_mapping = {
    'calling': 'normal',
    'clapping': 'normal',
    'cycling': 'normal',
    'dancing': 'normal',
    'drinking': 'normal',
    'eating': 'normal',
    'fighting': 'abnormal',
    'hugging': 'normal',
    'laughing': 'normal',
    'listening_to_music': 'normal',
    'running': 'normal',
    'sitting': 'normal',
    'sleeping': 'normal',
    'texting': 'normal',
    'using_laptop': 'normal',
    'unidentified': 'abnormal'
}

In [20]:
# Add 'type' column based on mapping
metadata_df['type'] = metadata_df['label'].map(label_type_mapping)

metadata_df.head()

,new_filename,label,source,type
0,Image_1.jpg,sitting,train,normal
1,Image_2.jpg,using_laptop,train,normal
2,Image_3.jpg,hugging,train,normal
3,Image_4.jpg,sleeping,train,normal
4,Image_5.jpg,using_laptop,train,normal


# Load Images

In [21]:
# Function to load image from GCS blob as array
def load_image_from_gcs(bucket, image_path, target_size=(224, 224)):
    blob = bucket.blob(image_path)
    img_bytes = blob.download_as_bytes()
    img = Image.open(BytesIO(img_bytes)).convert('RGB')
    img = img.resize(target_size)
    img_array = image.img_to_array(img)
    img_array = preprocess_input(img_array)
    return np.expand_dims(img_array, axis=0)

In [23]:
# Load a sample
sample_image_path = 'processed/svm_final_img/images/Image_1.jpg'
sample_image = load_image_from_gcs(bucket, sample_image_path)

print("Sample image shape:", sample_image.shape)

Sample image shape: (1, 224, 224, 3)


In [24]:
# Prepare new columns
metadata_df['vgg19_refined_label'] = ''
metadata_df['status'] = ''
metadata_df['final_label'] = ''

In [25]:
# Define known labels
known_labels = [
    'calling', 'clapping', 'cycling', 'dancing', 'drinking',
    'eating', 'fighting', 'hugging', 'laughing', 'listening_to_music',
    'running', 'sitting', 'sleeping', 'texting', 'using_laptop'
]

# Train Final Labelled HAR dataset (SVM output) with CNN+VGG19

In [26]:
# Load pretrained VGG19 base
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

# CNN1 classifier
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(metadata_df['label'].unique()), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg19 (Functional)              │ (None, 7, 7, 512)      │    20,024,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 15)             │         3,855 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,451,023 (100.90 MB)

 Trainable params: 6,426,639 (24.52 MB)

 Non-trainable params: 20,024,384 (76.39 MB)

In [28]:
vgg19_imagenet = VGG19(weights='imagenet')

In [34]:
# Create label mapping for index to label
label_mapping = {i: label for i, label in enumerate(metadata_df['label'].unique())}
reverse_mapping = {v: k for k, v in label_mapping.items()}

In [35]:
# Prediction + Refinement Function
def classify_with_refinement(img_array, cnn_model, imagenet_model, label_mapping, threshold=0.5):
    preds = cnn_model.predict(np.expand_dims(img_array, axis=0))[0]
    top_idx = np.argmax(preds)
    top_label = label_mapping[top_idx]
    confidence = preds[top_idx]

    if confidence >= threshold and top_label != 'unidentified':
        return top_label, 'existing'
    else:
        imagenet_preds = imagenet_model.predict(np.expand_dims(img_array, axis=0))
        decoded = decode_predictions(imagenet_preds, top=1)[0][0]
        imagenet_label = decoded[1]  # e.g. 'standing', 'smiling'

        if imagenet_label in metadata_df['label'].unique():
            return imagenet_label, 'existing'
        else:
            return imagenet_label, 'new'

In [36]:
# Loop through test images only
for idx, row in tqdm(metadata_df[metadata_df['source'] == 'test'].iterrows(), total=len(metadata_df[metadata_df['source'] == 'test'])):
    image_path = f"processed/svm_final_img/images/{row['new_filename']}"
    img_array = load_image_from_gcs(bucket, image_path)

    preds = vgg19_imagenet.predict(img_array)  # Use full pretrained ImageNet model
    decoded_preds = decode_predictions(preds, top=1)[0][0][1]  # Get top-1 prediction label

    metadata_df.loc[idx, 'vgg19_refined_label'] = decoded_preds

    if decoded_preds in known_labels:
        metadata_df.loc[idx, 'status'] = 'existing'
        metadata_df.loc[idx, 'final_label'] = decoded_preds
    else:
        metadata_df.loc[idx, 'status'] = 'new'
        metadata_df.loc[idx, 'final_label'] = decoded_preds

  0%|          | 0/5400 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


  0%|          | 1/5400 [00:01<2:18:06,  1.53s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


  0%|          | 2/5400 [00:01<1:14:33,  1.21it/s]

...

100%|█████████▉| 5398/5400 [33:47<00:00,  2.65it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


100%|█████████▉| 5399/5400 [33:47<00:00,  2.55it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


100%|██████████| 5400/5400 [33:48<00:00,  2.66it/s]


In [37]:
# For train images — retain original label as final_label and mark status existing
metadata_df.loc[metadata_df['source'] == 'train', 'final_label'] = metadata_df.loc[metadata_df['source'] == 'train', 'label']
metadata_df.loc[metadata_df['source'] == 'train', 'status'] = 'existing'

metadata_df.to_csv('cnn1_final_metadata.csv', index=False)



In [38]:
# Upload to GCS
metadata_blob = bucket.blob('metadata/cnn1_final_metadata.csv')
metadata_blob.upload_from_filename('cnn1_final_metadata.csv')

print("CNN1 final metadata uploaded to GCS at 'metadata/cnn1_final_metadata.csv'")

CNN1 final metadata uploaded to GCS at 'metadata/cnn1_final_metadata.csv'


In [39]:
cnn1_model = Sequential([
    VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(known_labels), activation='softmax')
])

cnn1_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [43]:
cnn1_model.save('cnn1_vgg19_model.h5')

model_blob = bucket.blob('models/cnn1_vgg19_model.h5')
model_blob.upload_from_filename('cnn1_vgg19_model.h5')

print("CNN1 model saved and uploaded to GCS at 'models/cnn1_vgg19_model.h5'")

cnn1_model.save('cnn1_vgg19_model.keras')
model_blob = bucket.blob('models/cnn1_vgg19_model.keras')
model_blob.upload_from_filename('cnn1_vgg19_model.keras')
print("CNN1 model saved and uploaded to GCS at 'models/cnn1_vgg19_model.keras'")

CNN1 model saved and uploaded to GCS at 'models/cnn1_vgg19_model.h5'
CNN1 model saved and uploaded to GCS at 'models/cnn1_vgg19_model.keras'


# Upload to GCS

In [44]:
!cp "/content/drive/MyDrive/Colab Notebooks/P2-05_CNN1-img.ipynb" "/content/P2-05_CNN1-img.ipynb"

!jupyter nbconvert --to script "/content/P2-05_CNN1-img.ipynb" --output "/content/P2-05_CNN1-img"
!mv /content/P2-05_CNN1-img.txt /content/P2-05_CNN1-img.py

client = storage.Client(project='p2-anomaly')
bucket = client.bucket('p2-anomaly')
notebook_blob = bucket.blob('notebooks/P2-05_CNN1-img.ipynb')
notebook_blob.upload_from_filename('/content/P2-05_CNN1-img.ipynb')
print("Notebook pushed to GCS.")

local_script_path = '/content/P2-05_CNN1-img.py'
gcs_script_path = 'scripts/P2-05_CNN1-img.py'
script_blob = bucket.blob(gcs_script_path)
script_blob.upload_from_filename(local_script_path)
print("Script uploaded to GCS.")

[NbConvertApp] Converting notebook /content/P2-05_CNN1-img.ipynb to script
[NbConvertApp] Writing 7424 bytes to /content/P2-05_CNN1-img.txt
Notebook pushed to GCS.
Script uploaded to GCS.


#Summary